In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


In [2]:
# Set the path to the train and test directories
train_data_dir = 'D:/data_split/train'
test_data_dir = 'D:/data_split/test'

# Set the number of classes and image dimensions
num_classes = 13
image_size = (96, 96)
batch_size = 8
class_names = [
    'Andorra_2019',
    'Common',
    'Lithuania_2021',
    'Monaco_2015',
    'Monaco_2016',
    'Monaco_2017',
    'Monaco_2018',
    'Monaco_2019',
    'SanMarino_2004',
    'SanMarino_2005',
    'Vatican_2004',
    'Vatican_2005',
    'Vatican_2006'
]


# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Normalization for testing/validation
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators for loading the train and test images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    classes = class_names,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42,
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    classes = class_names,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42,
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    classes = class_names,
    batch_size=batch_size,
    class_mode='categorical',
    #seed=42,
    shuffle=False
)



Found 5150 images belonging to 13 classes.
Found 1283 images belonging to 13 classes.
Found 724 images belonging to 13 classes.


In [3]:
# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Freeze the weights of the pre-trained layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Build the improved CNN model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set the number of training and validation steps per epoch
train_steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps_per_epoch = validation_generator.n // validation_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=100,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch
)

# Evaluate the model on the test dataset
test_steps_per_epoch = test_generator.n // test_generator.batch_size
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 256)               1179904   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 13)                3341      
                                                                 
Total params: 15,897,933
Trainable params: 1,183,245
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/100
643/643 [===============

643/643 [==============================] - 302s 470ms/step - loss: 0.1920 - accuracy: 0.9269 - val_loss: 0.7294 - val_accuracy: 0.8656
Epoch 50/100
643/643 [==============================] - 292s 455ms/step - loss: 0.1806 - accuracy: 0.9325 - val_loss: 0.6562 - val_accuracy: 0.8477
Epoch 51/100
643/643 [==============================] - 294s 458ms/step - loss: 0.1921 - accuracy: 0.9265 - val_loss: 0.7313 - val_accuracy: 0.8562
Epoch 52/100
643/643 [==============================] - 318s 495ms/step - loss: 0.1792 - accuracy: 0.9339 - val_loss: 0.7323 - val_accuracy: 0.8641
Epoch 53/100
643/643 [==============================] - 307s 478ms/step - loss: 0.2188 - accuracy: 0.9148 - val_loss: 0.7916 - val_accuracy: 0.8531
Epoch 54/100
643/643 [==============================] - 344s 535ms/step - loss: 0.1855 - accuracy: 0.9360 - val_loss: 0.7659 - val_accuracy: 0.8719
Epoch 55/100
643/643 [==============================] - 346s 538ms/step - loss: 0.1938 - accuracy: 0.9280 - val_loss: 0.6898 

In [ ]:
# Load the pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Freeze the weights of the pre-trained layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Build the improved CNN model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set the number of training and validation steps per epoch
train_steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps_per_epoch = validation_generator.n // validation_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=50,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch
)

# Evaluate the model on the test dataset
test_steps_per_epoch = test_generator.n // test_generator.batch_size
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
